In [ ]:
# !pip install scrapy

In [1]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

# init

In [ ]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    start_urls = ['http://www.imdb.com/chart/top',]
   
    def parse(self, response):
        # получение таблицы со строками, хараактеризующими ТОП-фильмы
        table_rows = response.xpath(
            './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
            'tbody[@class="lister-list"]/tr' # tr - обращение к строкам таблицы
        
        )
        
        for row in table_rows:
            # для каждой строки извлечем необходимую информацию
            yield {
                # пока парсим 3 колонки
                "title": row.xpath("./td[@class='titleColumn']/a/text()").extract_first(),
                "year": row.xpath("./td[@class='titleColumn']/span/text()").extract_first().strip("() "),
                "rating": row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
            }

In [ ]:
DEBUG_URL = 'http://www.imdb.com/chart/top'

req = requests.get(DEBUG_URL) # весь контент сайта
response = HtmlResponse(url=DEBUG_URL, body=req.content)# оборачиваем чтобы применить selector

selector = Selector(response=response)


In [ ]:
selector.xpath('.//*[@class="chart full-width" and @data-caller-name="chart-top250movie"]'
               '/tbody[@class="lister-list"]')[0].extract()

In [ ]:
# первый фильм
selector.xpath('.//*[@class="chart full-width" and @data-caller-name="chart-top250movie"]'
               '/tbody[@class="lister-list"]/tr')[0].extract()

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()

process.crawl(ImdbSpider)
process.start()

# Пример с обходом страниц

In [1]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [2]:
# dataclass

class MovieItem(scrapy.Item):
    title = scrapy.Field()
    rating = scrapy.Field()
    summary = scrapy.Field()
    genre = scrapy.Field()
    runtime = scrapy.Field()
    directors = scrapy.Field()
    writers = scrapy.Field()
    cast = scrapy.Field()

class ActorItem(scrapy.Item):
    actor_name = scrapy.Field()
    character = scrapy.Field()

In [ ]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    base_url = "https://imdb.com"
    start_urls = ['https://www.imdb.com/chart/top',]
   
    def parse(self, response):
        table_rows = response.xpath(
            './/*[@class="chart full-width" and @data-caller-name="chart-top250movie"]/'
            'tbody[@class="lister-list"]/tr'
        
        )
        for row in table_rows:
            rating = row.xpath("./td[@class='ratingColumn imdbRating']/strong/text()").extract_first(),
            rel_url = row.xpath("td[@class='titleColumn']/a/@href").extract_first().strip()
            row_url = self.base_url + rel_url
            yield scrapy.Request(row_url, callback=self.parseOneMovie, meta={'rating' : rating})
    
    def parseOneMovie(self, response):
        item = MovieItem()
        item["rating"] = response.meta["rating"][0]
        item['title'] = response.xpath('.//*/div[@class="title_wrapper"]/h1/text()').extract_first().strip()
        item["summary"] = response.xpath(".//*/div[@class='summary_text']/text()").extract_first().strip()
        item['directors'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract_first().strip()
        item['writers'] = response.xpath('.//*/div[@class="credit_summary_item"]/a/text()').extract()[1].strip()

        item["cast"] = list()

        for cast in response.xpath(".//table[@class='cast_list']/tr")[1:]:
            actor = ActorItem()
            
            actor["actor_name"] = cast.xpath("./td[2]/a/text()").extract_first().strip()
            actor["character"] = cast.xpath("./td[@class='character']/a/text()").extract_first()
            
            item["cast"].append(actor)

        return item

In [ ]:
from scrapy.crawler import CrawlerProcess

In [ ]:
process = CrawlerProcess()

process.crawl(ImdbSpider)
process.start()

In [ ]:
DEBUG_URL = 'http://www.imdb.com/title/tt0019254/'

req = requests.get(DEBUG_URL)
response = HtmlResponse(url=DEBUG_URL, body=req.content)

selector = Selector(response=response)


In [ ]:
selector.xpath(".//table[@class='cast_list']/tr")

# задание

jl - лучше обычного json, тк каждая строка валидный json объект - можно работать параллельно (ворекры)

In [1]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [2]:
class ActorItem(scrapy.Item):
    bio = scrapy.Field()
    born = scrapy.Field()
    movies = scrapy.Field()
    name = scrapy.Field()
    url = scrapy.Field()

In [3]:
class MovieItem(scrapy.Item):
    url = scrapy.Field()
    title = scrapy.Field()
    cast = scrapy.Field()

In [4]:
class ImdbSpider(scrapy.Spider):
    name = "imdb"
    allowed_domains = ["imdb.com"]
    base_url = "https://www.imdb.com"
    start_urls = ['https://www.imdb.com/search/name/?gender=male%2Cfemale&ref_=nv_cel_m',]
   
    def parse(self, response):
        table_rows = response.xpath(
            './/*[@class="lister-list"]/div[@class="lister-item mode-detail"]'
            
        )
        
        for row in table_rows:
            rel_url = row.xpath('./div[@class="lister-item-content"]/h3/a/@href').extract_first().strip()
            actor_url = self.base_url + rel_url + '/'
            yield scrapy.Request(actor_url, callback=self.parseOneActor, 
                                 meta={'actor_url' : actor_url}
                                )
            
    def parseOneActor(self, response):
        
            actor = ActorItem()
            
            actor["bio"] = response.xpath('//*[@id="name-bio-text"]/div/div').xpath('normalize-space()').extract_first().strip('See full bio »')
            actor["born"] = response.xpath('//*[@id="name-born-info"]/time/@datetime').extract_first()
                                    
            top15_movies = response.xpath('//*[@id="filmography"]/div[2]').xpath('div/b/a/text()').extract()[:15]
            actor["movies"] = top15_movies
            
            actor["name"] = response.xpath('//*[@id="name-overview-widget-layout"]/tbody/tr[1]/td/h1/span/text()').extract_first()
            
            actor["url"] = response.meta["actor_url"]
            
            top15_movie_links = response.xpath('//*[@id="filmography"]/div[2]/div/b/a/@href').extract()[:15]
            top15_movie_links = [self.base_url + m for m in top15_movie_links]
#             top15_movie_links = [m.replace('https://', 'https://www.') for m in top15_movie_links]
            
            for movie_link in top15_movie_links:
                yield scrapy.Request(url=movie_link, callback=self.parseOneMovie, meta={"movie_link":movie_link})
            
#             return actor
        
    def parseOneMovie(self, response):
        
        movie = MovieItem()
        
        movie['url'] = response.meta['movie_link']
        movie['title'] = response.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div[1]/h1/text()')\
                .extract_first().encode('utf8').decode('utf8')
        
        movie['cast'] = response.xpath('.//*[@class="StyledComponents__CastItemSummary-y9ygcu-9 fBAofn"]/a/text()').extract()
        
        yield movie

In [5]:
from scrapy.crawler import CrawlerProcess

In [6]:
process = CrawlerProcess(settings={
        "FEED_URI": "movies.jl"
    })

process.crawl(ImdbSpider)
process.start()

2021-07-27 06:41:14 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-07-27 06:41:14 [scrapy.utils.log] INFO: Versions: lxml 4.4.2.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.6.9 (default, Jan 26 2021, 15:33:00) - [GCC 8.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.1, Platform Linux-4.15.0-144-generic-x86_64-with-Ubuntu-18.04-bionic
2021-07-27 06:41:14 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-07-27 06:41:14 [scrapy.crawler] INFO: Overridden settings:
{}
2021-07-27 06:41:14 [scrapy.extensions.telnet] INFO: Telnet Password: 3e87cfbdf7984af8
2021-07-27 06:41:14 [py.warnings] WARNING: /home/atuthvatullin/environments/albert/lib/python3.6/site-packages/scrapy/extensions/feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more detail

2021-07-27 06:41:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt6343706/> (referer: https://www.imdb.com/name/nm1083404/)
2021-07-27 06:41:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt6111130/> (referer: https://www.imdb.com/name/nm6651162/)
2021-07-27 06:41:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt5516152/> (referer: https://www.imdb.com/name/nm1083404/)
2021-07-27 06:41:29 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt6343706/>
{'cast': ['Michael Bolton',
          'Sarah Shahi',
          'Adam Scott',
          'Matthew Kimbrough',
          'Jimmy Graham',
          'Grey Griffin',
          'Brooke Shields',
          'Janeane Garofalo',
          'Louie Anderson',
          'Sinbad',
          'Bob Saget',
          'Sarah Silverman',
          'Randall Park',
          'Andy Samberg',
          'Kenny G',
          'Fred Armisen',
          'Colt

2021-07-27 06:41:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt4270458/>
{'cast': ['Carla Gugino',
          'Luke Wilson',
          'Imogen Poots',
          'Rafe Spall',
          'Keisha Castle-Hughes',
          'Peter Cambor',
          'Machine Gun Kelly',
          'Ron White',
          'Christopher Backus',
          'Mark Allyn',
          'Jacqueline Byers',
          'Branscombe Richmond',
          'Tanc Sade',
          'Finesse Mitchell',
          'Catero Colbert',
          'Ethan Michael Mora',
          'Jack Yang',
          'Derek D. Dow'],
 'title': 'Гастролёры',
 'url': 'https://www.imdb.com/title/tt4270458/'}
2021-07-27 06:41:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9372038/> (referer: https://www.imdb.com/name/nm5584750/)
2021-07-27 06:41:30 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt1492966/>
{'cast': ['Louis C.K.',
          'Hadley Delany',
          'Ur

2021-07-27 06:41:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9327706/> (referer: https://www.imdb.com/name/nm2394794/)
2021-07-27 06:41:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt13610562/>
{'cast': ['Karen Gillan',
          'Pedro Pascal',
          'Keegan-Michael Key',
          'Leslie Mann',
          'Rob Delaney',
          'Fred Armisen',
          'David Duchovny',
          'Maria Bakalova',
          'Peter Serafinowicz',
          'Iris Apatow',
          'Vir Das',
          'Donna Air',
          'Samson Kayo',
          'Danielle Vitalis',
          'Ben Ashenden',
          'Nick Donald',
          'Harry Trevaldwyn',
          'David Cheung'],
 'title': 'The Bubble',
 'url': 'https://www.imdb.com/title/tt13610562/'}
2021-07-27 06:41:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt10648342/> (referer: https://www.imdb.com/name/nm2394794/)
2021-07-27 06:41:32 [scrapy.co

2021-07-27 06:41:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt4052050/> (referer: https://www.imdb.com/name/nm2088803/)
2021-07-27 06:41:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt4052050/>
{'cast': ['Yvonne Strahovski',
          'Anna Pniowsky',
          'Abigail Pniowsky',
          'Ryan McDonald',
          'Justin Bruening',
          'Julian Bailey',
          'Stephanie Costa'],
 'title': 'Кукловод',
 'url': 'https://www.imdb.com/title/tt4052050/'}
2021-07-27 06:41:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt3829266/> (referer: https://www.imdb.com/name/nm2088803/)
2021-07-27 06:41:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt3829266/>
{'cast': ['Boyd Holbrook',
          'Trevante Rhodes',
          'Jacob Tremblay',
          'Keegan-Michael Key',
          'Olivia Munn',
          'Sterling K. Brown',
          'Thomas Jane',
          '

...

2021-07-27 06:43:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt7449274/> (referer: https://www.imdb.com/name/nm2811823/)
2021-07-27 06:43:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt9894470/> (referer: https://www.imdb.com/name/nm2811823/)
2021-07-27 06:43:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt13363828/> (referer: https://www.imdb.com/name/nm2811823/)
2021-07-27 06:43:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt7449274/>
{'cast': ['Morgan Fairchild',
          'Corbin Bernsen',
          'Sierra McCormick',
          'Brett Hargrave',
          'Jake Legas',
          'Hal Dion',
          'Ben Leiataua',
          'Joy Yaholkovsky',
          'Ayla Carda',
          'Karen Deone',
          'Bre Garner',
          'Janet Lopez',
          'Gabriella Missimo',
          'Mary Madaline Roe',
          'June Schreiner',
          'Corbin Schweitz

2021-07-27 06:43:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt6905686/> (referer: https://www.imdb.com/name/nm3718007/)
2021-07-27 06:43:11 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt6905686/>
{'cast': ['Jonathan Majors',
          'Jurnee Smollett',
          'Courtney B. Vance',
          'Michael Kenneth Williams',
          'Aunjanue Ellis',
          'Wunmi Mosaku',
          'Abbey Lee',
          'Jamie Chung',
          'Jada Harris',
          'Jordan Patrick Smith',
          'Jamie Neumann',
          'Mac Brandt',
          'Alex Collins',
          'Joaquina Kalukango',
          'Jon Hudson Odom',
          'Erica Tazel',
          'Jonathon Pawlowski',
          'Deron J. Powell'],
 'title': 'Страна Лавкрафта',
 'url': 'https://www.imdb.com/title/tt6905686/'}
2021-07-27 06:43:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt10954600/> (referer: https://www.imdb.com/name/nm371

2021-07-27 06:43:13 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt0286106/>
{'cast': ['Mel Gibson',
          'Joaquin Phoenix',
          'Rory Culkin',
          'Abigail Breslin',
          'Cherry Jones',
          'M. Night Shyamalan',
          'Patricia Kalember',
          'Ted Sutton',
          'Merritt Wever',
          'Lanny Flaherty',
          'Marion McCorry',
          'Michael Showalter',
          'Kevin Pires',
          'Clifford David',
          'Rhonda Overby',
          'Greg Wood',
          'Paul L. Nolan',
          'Ukee Washington'],
 'title': 'Знаки',
 'url': 'https://www.imdb.com/title/tt0286106/'}
2021-07-27 06:43:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt3567288/> (referer: https://www.imdb.com/name/nm0796117/)
2021-07-27 06:43:13 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt3567288/>
{'cast': ['Olivia DeJonge',
          'Ed Oxenbould',
          'Deanna 

2021-07-27 06:43:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt11044858/> (referer: https://www.imdb.com/name/nm2858875/)
2021-07-27 06:43:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.imdb.com/title/tt7131622/> (referer: https://www.imdb.com/name/nm2858875/)
2021-07-27 06:43:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt11044858/>
{'cast': ['Sydney Sweeney',
          'Madison Iseman',
          'Jacques Colimon',
          'Ivan Shaw',
          'Julie Benz',
          'Rodney To',
          'JoNell Kennedy',
          'John Rothman',
          'Brandon Keener',
          'Miles McKenna',
          'Stephon Fuller',
          'A.J. Tannen',
          'Ji Eun Hwang',
          'Asia Jackson',
          'Phillip Wampler'],
 'title': 'Ноктюрн',
 'url': 'https://www.imdb.com/title/tt11044858/'}
2021-07-27 06:43:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt7131622/>
{'cast':

2021-07-27 06:43:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt4547938/>
{'cast': ['Olivia Cooke',
          'Christopher Abbott',
          'Mireille Enos',
          'Mary Steenburgen',
          'Keir Gilchrist',
          'Jim Belushi',
          'Chris Lowell',
          'Nate Corddry',
          'Natasha Bassett',
          'Alex Breaux',
          'Gene Jones',
          'MorningStar Angeline',
          'Eric Steinig',
          'Bernardo Saracino',
          'Mike Miller',
          'Morse Bicknell',
          'Sage Bell',
          'James Cady'],
 'title': 'Кэти уезжает',
 'url': 'https://www.imdb.com/title/tt4547938/'}
2021-07-27 06:43:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.imdb.com/title/tt3689484/>
{'cast': ['Keir Gilchrist',
          'Alex Wolff',
          'Imogen Poots',
          'Neve Campbell',
          'Tom Cullen',
          'Kiowa Gordon',
          'Kadrolsha Ona Carole',
          'Star Slade',
          

In [7]:
import json

data = []
with open('movies.jl') as f:
    for line in f:
        data.append(json.loads(line))
data

FileNotFoundError: [Errno 2] No such file or directory: 'movies.jl'

In [ ]:
for s in data:
    if s['name'] == 'Florence Pugh':
        print(s)

# ideal

In [ ]:
START_URL = 'https://www.imdb.com/search/name/?gender=male%2Cfemale&ref_=nv_cel_m'

BASE_URL = "https://www.imdb.com"

In [ ]:
def get_info_top_50(start_url, base_url=BASE_URL):
    req = requests.get(start_url)
    response = HtmlResponse(url=start_url, body=req.content)
    rows = response.xpath('//*[@id="main"]/div/div[@class="lister-list"]/div[@class="lister-item mode-detail"]')
    links = []

    for col in rows:
        rel_url = col.xpath("./div[@class='lister-item-content']/h3/a/@href").extract_first().strip()
        actor_url = base_url + rel_url + '/'
        links.append(actor_url)
    return links

In [ ]:
def get_info_about_one_actor(actor_url, base_url=BASE_URL, top_k_films=15):
    req = requests.get(actor_url)
    response = HtmlResponse(url=actor_url, body=req.content)
    name_widget = response.xpath('//*[@id="name-overview-widget-layout"]')
    item = dict()
    item['name'] = name_widget.xpath('./tbody/tr[1]/td/h1/span/text()').extract_first().strip()
    item['bio'] = ''.join(
        name_widget.xpath('//*[@id="name-bio-text"]/div/div//text()').extract()
    ).replace('See full bio »', ' ').strip()
    
    try:
        item['born'] = name_widget.xpath('//*[@id="name-born-info"]/time/@datetime').extract_first().strip()
    except AttributeError as e:
        item['born'] = None
    item['url'] = response.url
    films = response.xpath(
        '//*[@id="filmography"]/div[@class="filmo-category-section"][1]/div[contains(@class, "filmo-row")]'
    )
    item['movies'] = []
    item['movie_links'] = []
    for movie in films[:top_k_films]:
        rel_url = movie.xpath("./b[1]/a/@href").extract_first().strip()
        movie_url = base_url + rel_url
        ttl = movie.xpath("./b[1]/a/text()").extract_first().strip()
        item['movies'].append(ttl)
        item['movie_links'].append(movie_url)
    
    return item

In [ ]:
def get_info_about_one_movie(movie_url):
    req = requests.get(movie_url)
    response = HtmlResponse(url=movie_url, body=req.content)
    movie_name = response.xpath('//*[@class="title_wrapper"]/h1/text()').extract_first().strip()
    item = dict()
    item['url'] = movie_url
    item['title'] = movie_name
    cast = response.xpath('//*[@class="cast_list"]//tr//@title').extract()
    cast = [i.strip() for i in cast if i]
    item['cast'] = cast
    return item

In [ ]:
actor_links = get_info_top_50(START_URL)

In [ ]:
actor_data = [get_info_about_one_actor(cur_url) for cur_url in tqdm(actor_links)]

In [ ]:
flatten = lambda t: [item for sublist in t for item in sublist]
all_movies_links = flatten([cur_item['movie_links'] for cur_item in actor_data])

In [ ]:
movie_data = [get_info_about_one_movie(movie_url) for movie_url in tqdm(all_movies_links)]

In [ ]:
def dump_as_jl(inp_list_of_dicts, out_path):
    with open(out_path, 'w', encoding='utf-8') as fout:
        for line in inp_list_of_dicts:
            tmp_str = json.dumps(line, ensure_ascii=False) + '\n'
            fout.write(tmp_str)

In [ ]:
dump_as_jl(actor_data, 'actors.jl')

In [ ]:
dump_as_jl(movie_data, 'movies.jl')

# debug

In [2]:
import scrapy
from scrapy.http import HtmlResponse
from scrapy import Selector

import requests

In [3]:
# DEBUG_URL = 'https://www.imdb.com/name/nm6073955'
# DEBUG_URL = 'https://www.imdb.com/name/nm7326511/'

# DEBUG_URL = 'https://www.imdb.com/title/tt3281548/'
# DEBUG_URL = 'https://www.imdb.com/title/tt0437086/'
DEBUG_URL = "https://www.imdb.com/title/tt1442566/"

req = requests.get(DEBUG_URL) # весь контент сайта
response = HtmlResponse(url=DEBUG_URL, body=req.content)# оборачиваем чтобы применить selector

selector = Selector(response=response)

In [ ]:
selector.xpath('//*[@id="name-bio-text"]/div/div').xpath('normalize-space()').extract_first()

In [4]:
selector.xpath('//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div[1]/h1/text()')\
                .extract_first().encode('utf8').decode('utf8')

'Спецотдел по спасению'

In [ ]:
/

In [117]:
selector.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[1]/div[1]/div[1]/div[1]//img/@alt').extract()#[1]

['Saoirse Ronan']

In [3]:
selector.xpath('.//*[@class="StyledComponents__CastItemSummary-y9ygcu-9 fBAofn"]/a/text()').extract()

['Saoirse Ronan',
 'Emma Watson',
 'Florence Pugh',
 'Eliza Scanlen',
 'Laura Dern',
 'Timothée Chalamet',
 'Tracy Letts',
 'Bob Odenkirk',
 'James Norton',
 'Louis Garrel',
 'Jayne Houdyshell',
 'Chris Cooper',
 'Meryl Streep',
 'Rafael Silva',
 'Mason Alban',
 'Emily Edström',
 'Maryann Plunkett',
 'Hadley Robinson']

In [42]:
for s in selector.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]'):
    print(s.xpath('//img/@alt').extract())

In [50]:
'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]'

'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]'

In [49]:
selector.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]').extract()

['<section class="ipc-page-section ipc-page-section--base"><div data-testid="tmd_awards" class="Awards__Wrapper-sc-1qdt65t-0 bYVnIa"><div class="Awards__List-sc-1qdt65t-1 dGIciO"></div><ul class="ipc-metadata-list ipc-metadata-list--dividers-none Awards__Content-sc-1qdt65t-2 pWPhK ipc-metadata-list--base" role="presentation"><li role="presentation" class="ipc-metadata-list__item ipc-metadata-list-item--link" data-testid="award_information"><a class="ipc-metadata-list-item__label ipc-metadata-list-item__label--link" rel="" href="/title/tt0437086/awards/?ref_=tt_awd" target="">Awards</a><div class="ipc-metadata-list-item__content-container"><ul class="ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base" role="presentation"><li role="presentation" class="ipc-inline-list__item"><span class="ipc-metadata-list-item__list-content-item">10 wins &amp; 24 nominations</span></li></ul></div><a class="ipc-metadata-list-item__icon-link" hr

In [3]:
for s in selector.xpath('//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div'):
    print(s.xpath('div[1]/div[1]/div[1]//img/@alt').extract_first())

In [ ]:
l = []
for s in selector.xpath('//*[@id="filmography"]/div[2]'):
    l.append(s.xpath('//div/b/a/@href').extract())